In [ ]:
import requests
from datetime import datetime, timezone
import numpy as np
import matplotlib.pyplot as plt

ROOT_URL = "https://192.168.100.141/piwebapi"
COMPUTER = "WIN-5MGCA5NMS7S"
DATABASE = "Database1"
USERNAME = f"{COMPUTER}\Administrator"
PASSWORD = "abcd1234.."

In [ ]:
# Test Server
response_server = requests.get(f"{ROOT_URL}/", auth=(USERNAME, PASSWORD), verify=False)
response_server.json()

In [ ]:
# Get Database WebID
elementPath = f"\\\\{COMPUTER}\\{DATABASE}"
rsp_element_raw = requests.get(f"{ROOT_URL}/assetdatabases?path={elementPath}", auth=(USERNAME, PASSWORD), verify=False)
DATABASE_WEBID = rsp_element_raw.json()['WebId']

In [ ]:
# Get By Path Element
elementPath = f"\\\\{COMPUTER}\\{DATABASE}\\NuGreen\\Tucson\\Distilling Process\\Equipment\\F-110"
rsp_element_raw = requests.get(f"{ROOT_URL}/elements?path={elementPath}", auth=(USERNAME, PASSWORD), verify=False)
rsp_element_json = rsp_element_raw.json()

rsp_atr_raw = requests.get(rsp_element_json['Links']['Attributes'], auth=(USERNAME, PASSWORD), verify=False)
rsp_atr_json = rsp_atr_raw.json()
rsp_atr_json

In [ ]:
# Search By Elements
rsp_elmQuery_raw = requests.get(f"{ROOT_URL}/elements/search?databaseWebId={DATABASE_WEBID}&query=Name:=*b-2* Template:=Boiler", auth=(USERNAME, PASSWORD), verify=False)
rsp_elmQuery_json = rsp_elmQuery_raw.json() 
rsp_elmQuery_json

In [ ]:
# Search By Attributes
rsp_attrQuery_raw = requests.get(f"{ROOT_URL}/attributes/search?databaseWebId={DATABASE_WEBID}&query=Element:{{ Template:='Unit' }} Name:='*flow*' Description:='*flow rate*'", auth=(USERNAME, PASSWORD), verify=False)
rsp_attrQuery_json = rsp_attrQuery_raw.json() 
rsp_attrQuery_json

In [ ]:
# Get Stream Data
rsp_strm_raw = requests.get(f"{ROOT_URL}/streams/{rsp_attrQuery_json['Items'][0]['WebId']}/recorded", auth=(USERNAME, PASSWORD), verify=False)
rsp_strm_json = rsp_strm_raw.json() 

# And Plot It
data_recorded = np.array([])
for data_now in rsp_strm_json['Items']:
    try:
        float(data_now['Value'])
        data_recorded = np.append(data_recorded, data_now['Value'])

        # datetime.fromisoformat('2024-04-28T21:09:04Z'[:-1])
    except:
        data_recorded = np.append(data_recorded, -1)

fig, ax = plt.subplots()
ax.plot(data_recorded)
plt.show()